In [0]:
import os
import sys
import shutil
import argparse
import tempfile
import urllib.request
import zipfile
import json

In [2]:
!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

Cloning into 'bert_repo'...
remote: Enumerating objects: 333, done.
remote: Total 333 (delta 0), reused 0 (delta 0), pack-reused 333
Receiving objects: 100% (333/333), 282.45 KiB | 3.77 MiB/s, done.
Resolving deltas: 100% (183/183), done.



In [3]:
! git clone https://github.com/gsasikiran/temp_data training_data

Cloning into 'training_data'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [4]:
! wget https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip 

--2019-10-02 12:14:34--  https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 2607:f8b0:4001:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1248381879 (1.2G) [application/zip]
Saving to: ‘wwm_uncased_L-24_H-1024_A-16.zip’

wwm_uncased_L-24_H- 100%[===================>]   1.16G  76.1MB/s    in 11s     

2019-10-02 12:14:45 (113 MB/s) - ‘wwm_uncased_L-24_H-1024_A-16.zip’ saved [1248381879/1248381879]



In [0]:
with zipfile.ZipFile('/content/wwm_uncased_L-24_H-1024_A-16.zip', 'r') as f:
  f.extractall('/content/BERT_weights')

In [6]:
!python /content/bert_repo/create_pretraining_data.py \
  --input_file=/content/training_data/training_data.txt\
  --output_file=/tmp/tf_examples.tfrecord \
  --vocab_file=/content/training_data/deep_vocab.txt \
  --do_lower_case=True \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=12345 \
  --dupe_factor=5



W1002 12:15:26.905479 140228498020224 deprecation_wrapper.py:119] From /content/bert_repo/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1002 12:15:26.905869 140228498020224 deprecation_wrapper.py:119] From /content/bert_repo/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1002 12:15:26.906005 140228498020224 deprecation_wrapper.py:119] From /content/bert_repo/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1002 12:15:26.924406 140228498020224 deprecation_wrapper.py:119] From /content/bert_repo/create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W1002 12:15:26.929473 140228498020224 deprecation_wrapper.py:119] From /content/bert_repo/create_pretraining_data.py:446: The name tf.logging.info is deprecated. Please us

In [7]:
!python /content/bert_repo/run_pretraining.py \
  --input_file=/tmp/tf_examples.tfrecord \
  --output_dir=/content/pretraining_output \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=/content/BERT_weights/wwm_uncased_L-24_H-1024_A-16/bert_config.json \
  --train_batch_size=32 \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --num_train_steps=20 \
  --num_warmup_steps=10 \
  --learning_rate=2e-4 \
  --num_train_steps=2 \
  --iterations_per_loop=1




W1002 12:16:03.701385 140554317916032 deprecation_wrapper.py:119] From /content/bert_repo/run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1002 12:16:03.701612 140554317916032 deprecation_wrapper.py:119] From /content/bert_repo/run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1002 12:16:03.701754 140554317916032 deprecation_wrapper.py:119] From /content/bert_repo/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1002 12:16:03.702365 140554317916032 deprecation_wrapper.py:119] From /content/bert_repo/run_pretraining.py:414: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W1002 12:16:03.702681 140554317916032 deprecation_wrapper.py:119] From /content/bert_repo/run_pretraining.py:418: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.




In [8]:
# create dummy data to get embeddings
!echo 'Who was Jim Henson ?' > /tmp/input.txt

# get embeddings for the dummy data
!python /content/bert_repo/extract_features.py \
  --input_file=/tmp/input.txt\
  --output_file=/tmp/output.jsonl \
  --vocab_file=/content/training_data/deep_vocab.txt \
  --bert_config_file=/content/BERT_weights/wwm_uncased_L-24_H-1024_A-16/bert_config.json \
  --init_checkpoint=/content/pretraining_output/model.ckpt-0 \
  --layers=-1 \
  --max_seq_length=128 \
  --batch_size=8



W1002 12:19:36.385470 140305768384384 deprecation_wrapper.py:119] From /content/bert_repo/extract_features.py:344: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1002 12:19:36.385682 140305768384384 deprecation_wrapper.py:119] From /content/bert_repo/extract_features.py:344: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1002 12:19:36.385808 140305768384384 deprecation_wrapper.py:119] From /content/bert_repo/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

I1002 12:19:38.337397 140305768384384 utils.py:141] NumExpr defaulting to 2 threads.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend

In [9]:
# view the output of that 1 example we had in the dummy data
import json
embedding = []
with open('/tmp/output.jsonl') as f:
    for line in f:
        embedding.append(json.loads(line))

features = embedding[0]["features"]

for feature in features:
  token_text = feature["token"]
  token_embedding = feature["layers"][0]["values"]
  
  print(f"token: {token_text}")
  print(f"embedding: {token_embedding[:10]}")
  print("\n")

token: [CLS]
embedding: [0.715035, -0.080827, 0.393635, 1.798277, -0.086014, 1.358826, -1.784077, 0.297213, -1.916145, 1.317705]


token: who
embedding: [0.295444, 0.609228, 0.816393, 0.900159, 0.055584, 1.537693, -1.063984, 0.580213, -0.789018, 0.852379]


token: was
embedding: [0.804743, -0.194332, 0.406966, 1.599644, 0.104317, 1.439876, -1.334046, -0.237154, -1.127866, 0.975077]


token: j
embedding: [-0.085923, 0.541564, 0.712147, 1.071972, 0.661168, 1.569185, -1.276443, 0.167014, -0.637366, 1.255047]


token: ##im
embedding: [-0.219272, -0.073292, 1.252351, 1.442163, 0.106982, 1.051385, -0.790406, 0.182098, -0.626894, 0.857871]


token: he
embedding: [0.373328, 0.211023, 0.503679, 1.301739, 0.01345, 1.501631, -1.048431, -0.082812, -1.419353, 0.603227]


token: ##ns
embedding: [0.518285, 1.002582, 1.099799, 1.258302, 0.289224, 1.430837, -0.511594, 0.352258, -0.6069, 1.070495]


token: ##on
embedding: [0.104986, 0.585657, 1.130982, 1.794896, 0.701496, 1.493641, -1.504521, 0.586159, 